In [4]:
import os
import pandas as pd

# Ruta de la carpeta que contiene los CSVs
ruta_carpeta = './'

# Cargar todos los archivos CSV
dfs = []
for archivo in os.listdir(ruta_carpeta):
    if archivo.endswith('.csv'):
        df = pd.read_csv(os.path.join(ruta_carpeta, archivo))
        df['Fuente'] = archivo  # Agregamos una columna para identificar el origen
        dfs.append(df)

# Combinar todos los DataFrames en uno solo
df_todos = pd.concat(dfs, ignore_index=True)


In [9]:
import pandas as pd
import os

def load_results(filename):
    """
    Carga el CSV y extrae el número de intervalo a partir de la columna 'Intervalos Acumulados'.
    Se asume que la cadena tiene el formato 'Intervalos 1 a X'.
    """
    df = pd.read_csv(filename)
    # Extraemos el número final del intervalo para usarlo como índice
    df['Intervalo'] = df['Intervalos Acumulados'].str.extract(r'a (\d+)').astype(int)
    df.set_index('Intervalo', inplace=True)
    return df

def obtener_tabla(horizonte):
    # Diccionario para almacenar DataFrames con sus nombres
    tablas = {}

    # Nombres de archivos para LSTM
    lstm_files = {
        'LSTM-7': f"lstm_7_{horizonte}.csv",
        'LSTM-14': f"lstm_14_{horizonte}.csv",
        'LSTM-21': f"lstm_21_{horizonte}.csv"
    }
    
    # Cargar cada CSV de LSTM y almacenar la métrica (por ejemplo, SMAPE)
    for key, file in lstm_files.items():
        if os.path.exists(file):
            df = load_results(file)
            # Extraer la columna de interés (por ejemplo, SMAPE)
            tablas[key] = df['SMAPE']
        else:
            raise FileNotFoundError(f"Archivo no encontrado: {file}")
    
    # Cargar la tabla de timesfm
    timesfm_file = f"timesfm_{horizonte}.csv"
    if not os.path.exists(timesfm_file):
        raise FileNotFoundError(f"Archivo no encontrado: {timesfm_file}")
    df_timesfm = load_results(timesfm_file)
    tablas['TimesFM'] = df_timesfm['SMAPE']

    # Combinar todas las series en un DataFrame
    tabla_resumen = pd.DataFrame(tablas)
    tabla_resumen.index.name = 'Intervalo'
    
    return tabla_resumen

# Obtener la tabla para horizonte 15
tabla_15 = obtener_tabla(15)
print("Tabla para horizonte 15:")
print(tabla_15)
# Si quieres guardarla en CSV
tabla_15.to_csv('tabla_horizonte_15.csv')

# Obtener la tabla para horizonte 30
tabla_30 = obtener_tabla(30)
print("\nTabla para horizonte 30:")
print(tabla_30)
tabla_30.to_csv('tabla_horizonte_30.csv')


Tabla para horizonte 15:
               LSTM-7     LSTM-14    LSTM-21     TimesFM
Intervalo                                               
1           28.994249   16.716748  20.461552   49.960816
2           19.427937   20.486859  23.319860   41.527691
3           28.384518   28.233642  29.396235   36.126297
4           50.134744   34.324984  27.063151   74.469446
5           48.055877   26.462351  26.622346   31.861175
6           14.375691   13.297392  47.368574  198.587984
7          105.960527   49.063209  42.008538  200.000000
8           94.609504  127.046611  94.896765  200.000000

Tabla para horizonte 30:
               LSTM-7     LSTM-14    LSTM-21     TimesFM
Intervalo                                               
1           21.073901   17.355785  17.436183  106.295114
2           17.385996   20.105861  18.863253   49.261507
3           29.649534   25.736964  27.576931   45.571056
4          187.538584   44.411005  18.523575  194.287409
5           28.981870   33.342295  24